In [4]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_percentage_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import LeaveOneOut
from sklearn.linear_model import LinearRegression
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.utils import resample

# Load the data
data = pd.read_csv('./data250824.csv', index_col=[0], header=[0])

In [5]:
# Define feature sets
feature_sets = {
    'vi-qc': ['X3', 'X2', 'X1', 'X17'],
    'ga-qc': ['X2', 'X4', 'X15', 'X16'],
    'vi-ms': ['M43', 'M58', 'M2', 'M42'],
    'ga-ms': ['M2', 'M52', 'M67', 'M102']
}

# Define models with tuned hyperparameters (FOR WHOLE DATA SET)
tuned_hyperparameters = {
    'vi-qc': {
        'RF': {'bootstrap': False, 'max_depth': None, 'max_features': 1.0, 'min_samples_leaf': 10, 'min_samples_split': 11, 'n_estimators': 99, 'random_state': 42, 'n_jobs': -1},
        'SVR': {'C': 124.34299606015858, 'epsilon': 0.1436663234855287, 'kernel': 'rbf'},
        'XGB': {'booster':'gbtree', 'device':'cpu', 'objective':'reg:squarederror', 'verbosity': 2, 'tree_method':'auto', 'seed': 42, 'n_jobs': -1,
                'colsample_bytree': 0.10108222695147716, 'gamma': 0.03195015716291206, 'learning_rate': 0.1532942849179548, 'max_depth': 3, 'min_child_weight': 2.5672988836952646, 'reg_alpha': 0.28393875667236645, 'reg_lambda': 8.471227306156468, 'subsample': 0.36934305158488395}
    },
    'ga-qc': {
        'RF': {'bootstrap': True, 'max_depth': None, 'max_features': 1.0, 'min_samples_leaf': 1, 'min_samples_split': 6, 'n_estimators': 59, 'random_state': 42, 'n_jobs': -1},
        'SVR': {'C': 1.0, 'epsilon': 0.1, 'kernel': 'rbf'},
        'XGB': {'booster':'gbtree', 'objective':'reg:squarederror', 'verbosity': 2, 'tree_method':'auto', 'seed': 42, 'n_jobs': -1,
                'colsample_bytree': 0.1360716562727799, 'gamma': 0.16137390404554144, 'learning_rate': 0.2727409061252117, 'max_depth': 2, 'min_child_weight': 3.9511645755552687, 'reg_alpha': 0.16804646499331374, 'reg_lambda': 8.104479048297152, 'subsample': 0.6070548406804517},
        'MLR': {}
    },
    'vi-ms': {
        'RF': {'bootstrap': True, 'max_depth': None, 'max_features': 1.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 147, 'random_state': 42, 'n_jobs': -1},
        'SVR': {'C': 1.6028197376373123, 'degree': 4, 'epsilon': 0.0, 'kernel': 'rbf'},
        'XGB': {'booster':'gbtree', 'device':'cpu', 'objective':'reg:squarederror', 'verbosity': 2, 'tree_method':'auto', 'seed': 42, 'validate_parameters': True, 'n_jobs': -1,
                'colsample_bytree': 0.9915139189418589, 'gamma': 0.7183487649345512, 'learning_rate': 0.04234226319867907, 'max_depth': 6, 'min_child_weight': 0.7111973058597089, 'reg_alpha': 0.002473915667967762, 'reg_lambda': 0.2952577589021902, 'subsample': 0.33404173513149854}
    },
    'ga-ms': {
        'RF': {'bootstrap': True, 'max_depth': None, 'max_features': 1.0, 'min_samples_leaf': 1, 'min_samples_split': 3, 'n_estimators': 126, 'random_state': 42, 'n_jobs': -1},
        'SVR': {'C': 4.752986507554117, 'epsilon': 0.21223832828344527, 'kernel': 'rbf'},
        'XGB': {'booster':'gbtree', 'device':'cpu', 'objective':'reg:squarederror', 'verbosity': 2, 'tree_method':'auto', 'seed': 42, 'validate_parameters': True, 'n_jobs': -1,
                },
        'MLR': {}
    }
}

scaler = StandardScaler()
# Target variable
y = data['pIC50']

In [14]:
#FOR TRAINING SET
from sklearn.utils import resample

n_bootstraps = 1000
boot_results = []
mlr_coefs_collection = []
all_metrics_records = []  # ✅ New list to store per-bootstrap metrics

scaler = StandardScaler()

for feature_set_name, features in feature_sets.items():
    X_train = data[features].iloc[:22].values
    y_train = y.iloc[:22].values

    for model_name, hyperparams in tuned_hyperparameters[feature_set_name].items():
        if model_name == 'RF':
            model = RandomForestRegressor(**hyperparams)
            scale = False
        elif model_name == 'SVR':
            model = SVR(**hyperparams)
            scale = True
        elif model_name == 'XGB':
            model = XGBRegressor(**hyperparams)
            scale = False
        elif model_name == 'MLR':
            model = LinearRegression()
            scale = False

        metrics = {'r2': [], 'rmse': []}
        coefs = []
        intercepts = []

        for i in range(n_bootstraps):
            X_boot, y_boot = resample(X_train, y_train)

            if scale:
                X_boot_scaled = scaler.fit_transform(X_boot)
                X_train_scaled = scaler.transform(X_train)
                model.fit(X_boot_scaled, y_boot)
                y_pred = model.predict(X_train_scaled)
            else:
                model.fit(X_boot, y_boot)
                y_pred = model.predict(X_train)
                if model_name == 'MLR':
                    coefs.append(model.coef_)
                    intercepts.append(model.intercept_)  # ✅ Save intercept
            
            r2 = r2_score(y_train, y_pred)
            rmse = np.sqrt(mean_squared_error(y_train, y_pred))

            metrics['r2'].append(r2)
            metrics['rmse'].append(rmse)

            # ✅ Append per-bootstrap metric result
            all_metrics_records.append({
                'Feature Set': feature_set_name,
                'Model': model_name,
                'Bootstrap': i,
                'R2': r2,
                'RMSE': rmse
            })

        result = {
            'Feature Set': feature_set_name,
            'Model': model_name,
            'R2 Mean': np.mean(metrics['r2']),
            'R2 Std': np.std(metrics['r2']),
            'RMSE Mean': np.mean(metrics['rmse']),
            'RMSE Std': np.std(metrics['rmse']),
        }

        if model_name == 'MLR':
            # Save all individual coefficients and intercepts
            coef_df = pd.DataFrame(coefs, columns=features)
            coef_df['Intercept'] = intercepts  # ✅ Add intercept column
            coef_df['Feature Set'] = feature_set_name
            mlr_coefs_collection.append(coef_df)

        boot_results.append(result)

# ✅ Save summary metrics
results_df = pd.DataFrame(boot_results)
results_df.to_csv('./bootstrapping_eval_metrics_training.csv', index=False)
print(results_df)

# ✅ Save all bootstrap metrics
metrics_df = pd.DataFrame(all_metrics_records)
metrics_df.to_csv('./bootstrapped_all_metrics.csv', index=False)

# ✅ Save all MLR coefficients
if mlr_coefs_collection:
    all_mlr_coefs_df = pd.concat(mlr_coefs_collection, ignore_index=True)
    all_mlr_coefs_df.to_csv('./bootstrapped_mlr_coefficients.csv', index=False)


   Feature Set Model   R2 Mean    R2 Std  RMSE Mean  RMSE Std
0        vi-qc    RF  0.679058  0.148348   0.842178  0.188834
1        vi-qc   SVR  0.797652  0.173722   0.660444  0.182959
2        vi-qc   XGB  0.843489  0.055205   0.596184  0.088515
3        ga-qc    RF  0.898543  0.038073   0.478465  0.080982
4        ga-qc   SVR  0.849571  0.045533   0.584615  0.085893
5        ga-qc   XGB  0.851972  0.063384   0.575967  0.108822
6        ga-qc   MLR  0.886959  0.039051   0.507394  0.070181
7        vi-ms    RF  0.909801  0.057402   0.439669  0.126675
8        vi-ms   SVR  0.860346  0.071991   0.554435  0.129403
9        vi-ms   XGB  0.846827  0.080160   0.581403  0.132259
10       ga-ms    RF  0.853346  0.073353   0.569187  0.128130
11       ga-ms   SVR  0.859606  0.065439   0.556472  0.127280
12       ga-ms   XGB  0.853226  0.091514   0.559110  0.167527
13       ga-ms   MLR  0.900775  0.031752   0.475884  0.061969


In [15]:
#FOR TEST SET

n_bootstraps = 1000
boot_results = []
all_metrics_records = []  # ✅ Store per-bootstrap results

scaler = StandardScaler()
rng = np.random.default_rng()  # Make sure RNG is defined

for feature_set_name, features in feature_sets.items():
    X_train = data[features].iloc[:22].values
    y_train = y.iloc[:22].values

    X_test = data[features].iloc[22:28].values
    y_test = y.iloc[22:28].values

    for model_name, hyperparams in tuned_hyperparameters[feature_set_name].items():
        if model_name == 'RF':
            model = RandomForestRegressor(**hyperparams)
            scale = False
        elif model_name == 'SVR':
            model = SVR(**hyperparams)
            scale = True
        elif model_name == 'XGB':
            model = XGBRegressor(**hyperparams)
            scale = False
        elif model_name == 'MLR':
            model = LinearRegression()
            scale = False

        metrics = {'r2': [], 'rmse': []}

        # Train once on full training data
        if scale:
            X_train_scaled = scaler.fit_transform(X_train)
            model.fit(X_train_scaled, y_train)
            X_test_scaled = scaler.transform(X_test)
        else:
            model.fit(X_train, y_train)

        for i in range(n_bootstraps):
            X_boot, y_boot = resample(X_test, y_test, random_state=rng.integers(0, 1e6))

            if scale:
                y_pred = model.predict(scaler.transform(X_boot))
            else:
                y_pred = model.predict(X_boot)

            r2 = r2_score(y_boot, y_pred)
            rmse = np.sqrt(mean_squared_error(y_boot, y_pred))

            metrics['r2'].append(r2)
            metrics['rmse'].append(rmse)

            # ✅ Save individual bootstrap result
            all_metrics_records.append({
                'Feature Set': feature_set_name,
                'Model': model_name,
                'Bootstrap': i,
                'R2': r2,
                'RMSE': rmse
            })

        boot_results.append({
            'Feature Set': feature_set_name,
            'Model': model_name,
            'R2 Mean': np.mean(metrics['r2']),
            'R2 Std': np.std(metrics['r2']),
            'RMSE Mean': np.mean(metrics['rmse']),
            'RMSE Std': np.std(metrics['rmse'])
        })

# ✅ Save summary results
results_df = pd.DataFrame(boot_results)
results_df.to_csv('./bootstrapping_eval_metrics_test.csv', index=False)
print(results_df)

# ✅ Save all individual bootstrap results
metrics_df = pd.DataFrame(all_metrics_records)
metrics_df.to_csv('./bootstrapped_all_metrics_test.csv', index=False)


   Feature Set Model   R2 Mean     R2 Std  RMSE Mean  RMSE Std
0        vi-qc    RF  0.299637   4.481071   0.688038  0.239556
1        vi-qc   SVR  0.905175   0.295517   0.307279  0.069566
2        vi-qc   XGB -0.118110   7.239649   0.874901  0.320571
3        ga-qc    RF  0.939620   0.208517   0.234014  0.079401
4        ga-qc   SVR  0.906578   0.415638   0.272189  0.080926
5        ga-qc   XGB  0.694506   1.878848   0.403690  0.137822
6        ga-qc   MLR -0.152965  22.814389   0.451665  0.089268
7        vi-ms    RF  0.779058   1.034794   0.422757  0.147735
8        vi-ms   SVR  0.130907   8.937917   0.538784  0.167166
9        vi-ms   XGB  0.250703   3.053004   0.788631  0.318567
10       ga-ms    RF  0.686549   3.565466   0.313499  0.089890
11       ga-ms   SVR  0.935113   0.252058   0.256122  0.099720
12       ga-ms   XGB  0.849361   3.067491   0.213711  0.036915
13       ga-ms   MLR -0.433482  22.533211   0.262163  0.026137


In [6]:
from sklearn.base import clone
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

def bootstrap_prediction_intervals(
    X_train, y_train, X_test, model, n_bootstraps=1000, ci=95):
    
    preds = np.zeros((n_bootstraps, len(X_test)))
    residual_stds = []

    for i in range(n_bootstraps):
        # Resample
        X_res, y_res = resample(X_train, y_train)
        model_clone = clone(model)
        model_clone.fit(X_res, y_res)

        # Predict on test
        preds[i] = model_clone.predict(X_test)

        # Residuals on bootstrap train
        y_pred_train = model_clone.predict(X_res)
        residuals = y_res - y_pred_train
        residual_stds.append(np.std(residuals, ddof=1))

    # Final predicted mean
    mean_preds = np.mean(preds, axis=0)
    
    # Compute lower and upper prediction interval bounds
    alpha = (100 - ci) / 2
    lower_percentile = np.percentile(preds, alpha, axis=0)
    upper_percentile = np.percentile(preds, 100 - alpha, axis=0)

    # Add residual noise (pointwise)
    avg_residual_std = np.mean(residual_stds)
    lower_bound = lower_percentile - avg_residual_std
    upper_bound = upper_percentile + avg_residual_std

    return mean_preds, lower_bound, upper_bound

n_bootstraps = 1000
ci = 95

# Split datasets
X_train_full = data.iloc[:22]
y_train = y.iloc[:22].values
X_test_full = data.iloc[22:28]
y_test = y.iloc[22:28].values
X_val_full = data.iloc[28:42]
y_val = y.iloc[28:42].values

scaler = StandardScaler()

for feature_set_name, features in feature_sets.items():
    X_train = X_train_full[features].values
    X_test = X_test_full[features].values
    X_val = X_val_full[features].values

    for model_name, hyperparams in tuned_hyperparameters[feature_set_name].items():
        if model_name == 'RF':
            model = RandomForestRegressor(**hyperparams)
            scale = False
        elif model_name == 'SVR':
            model = SVR(**hyperparams)
            scale = True
        elif model_name == 'XGB':
            model = XGBRegressor(**hyperparams)
            scale = False
        elif model_name == 'MLR':
            model = LinearRegression()
            scale = False

        if scale:
            X_train_scaled = scaler.fit_transform(X_train)
            X_test_scaled = scaler.transform(X_test)
            X_val_scaled = scaler.transform(X_val)
        else:
            X_train_scaled = X_train
            X_test_scaled = X_test
            X_val_scaled = X_val

        # Compute prediction intervals for test set
        test_preds, test_lower, test_upper = bootstrap_prediction_intervals(
            X_train_scaled, y_train, X_test_scaled, model, n_bootstraps=n_bootstraps, ci=ci
        )

        # Compute prediction intervals for validation set
        val_preds, val_lower, val_upper = bootstrap_prediction_intervals(
            X_train_scaled, y_train, X_val_scaled, model, n_bootstraps=n_bootstraps, ci=ci
        )

        # Save test results
        test_df = pd.DataFrame({
            'y_true': y_test,
            'y_pred': test_preds,
            'lower_bound': test_lower,
            'upper_bound': test_upper
        })
        test_df['Feature Set'] = feature_set_name
        test_df['Model'] = model_name
        #test_df.to_csv(f'./pred_interval_test_{feature_set_name}_{model_name}.csv', index=False)

        # Save validation results
        val_df = pd.DataFrame({
            'y_true': y_val,
            'y_pred': val_preds,
            'lower_bound': val_lower,
            'upper_bound': val_upper
        })
        val_df['Feature Set'] = feature_set_name
        val_df['Model'] = model_name
        val_df.to_csv(f'./pred_interval_val_{feature_set_name}_{model_name}.csv', index=False)

        print(f"Saved prediction intervals for {model_name} with {feature_set_name}.")


Saved prediction intervals for RF with vi-qc.
Saved prediction intervals for SVR with vi-qc.
Saved prediction intervals for XGB with vi-qc.
Saved prediction intervals for RF with ga-qc.
Saved prediction intervals for SVR with ga-qc.
Saved prediction intervals for XGB with ga-qc.
Saved prediction intervals for MLR with ga-qc.
Saved prediction intervals for RF with vi-ms.
Saved prediction intervals for SVR with vi-ms.
Saved prediction intervals for XGB with vi-ms.
Saved prediction intervals for RF with ga-ms.
Saved prediction intervals for SVR with ga-ms.
Saved prediction intervals for XGB with ga-ms.
Saved prediction intervals for MLR with ga-ms.


In [18]:
from math import sqrt
from sklearn.model_selection import LeaveOneOut

n_bootstraps = 1000
rng = np.random.RandomState(42)

boot_records = []

for feature_set_name, features in feature_sets.items():
    X = data[features]
    X_orig = X.iloc[:22]
    y_orig = y.iloc[:22]

    # Compute Total Sum of Squares (TSS) once on original y
    y_mean = np.mean(y_orig)
    tss_orig = np.sum((y_orig - y_mean) ** 2)

    for model_name, hyperparams in tuned_hyperparameters.get(feature_set_name, {}).items():
        for b in range(n_bootstraps):
            # Bootstrap sampling
            X_boot, y_boot = resample(X_orig, y_orig, random_state=rng)

            y_true = []
            y_pred = []
            e_square = []

            loo = LeaveOneOut()

            for train_index, test_index in loo.split(X_boot):
                X_train_cv, X_test_cv = X_boot.iloc[train_index], X_boot.iloc[test_index]
                y_train_cv, y_test_cv = y_boot.iloc[train_index], y_boot.iloc[test_index]

                if model_name == 'RF':
                    model = RandomForestRegressor(**hyperparams)
                elif model_name == 'SVR':
                    scaler = StandardScaler()
                    X_train_cv = scaler.fit_transform(X_train_cv)
                    X_test_cv = scaler.transform(X_test_cv)
                    model = SVR(**hyperparams)
                elif model_name == 'XGB':
                    model = XGBRegressor(**hyperparams)
                elif model_name == 'MLR':
                    model = LinearRegression()

                model.fit(X_train_cv, y_train_cv)
                prediction = model.predict(X_test_cv)[0]

                error_sq = (y_test_cv.values[0] - prediction) ** 2
                y_true.append(y_test_cv.values[0])
                y_pred.append(prediction)
                e_square.append(error_sq)

            press = np.sum(e_square)
            r2_cv = 1 - (press / tss_orig)
            rmse_cv = np.sqrt(np.mean(e_square))

            boot_records.append({
                'Feature Set': feature_set_name,
                'Model': model_name,
                'Bootstrap': b + 1,
                'R2CV': r2_cv,
                'RMSECV': rmse_cv
            })

# Convert to DataFrame and save
boot_df = pd.DataFrame(boot_records)
boot_df.to_csv('./bootstrapped_loocv_results.csv', index=False)
print(boot_df.head())


  Feature Set Model  Bootstrap      R2CV    RMSECV
0       vi-qc    RF          1  0.675376  0.868025
1       vi-qc    RF          2  0.645618  0.906938
2       vi-qc    RF          3  0.847641  0.594670
3       vi-qc    RF          4  0.726399  0.796894
4       vi-qc    RF          5  0.470066  1.109055


In [19]:
#prediction interval for train

from sklearn.base import clone
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

def bootstrap_prediction_intervals(
    X_train, y_train, X_test, model, n_bootstraps=1000, ci=95):
    
    preds = np.zeros((n_bootstraps, len(X_test)))
    residual_stds = []

    for i in range(n_bootstraps):
        # Resample
        X_res, y_res = resample(X_train, y_train)
        model_clone = clone(model)
        model_clone.fit(X_res, y_res)

        # Predict on test
        preds[i] = model_clone.predict(X_test)

        # Residuals on bootstrap train
        y_pred_train = model_clone.predict(X_res)
        residuals = y_res - y_pred_train
        residual_stds.append(np.std(residuals, ddof=1))

    # Final predicted mean
    mean_preds = np.mean(preds, axis=0)
    
    # Compute lower and upper prediction interval bounds
    alpha = (100 - ci) / 2
    lower_percentile = np.percentile(preds, alpha, axis=0)
    upper_percentile = np.percentile(preds, 100 - alpha, axis=0)

    # Add residual noise (pointwise)
    avg_residual_std = np.mean(residual_stds)
    lower_bound = lower_percentile - avg_residual_std
    upper_bound = upper_percentile + avg_residual_std

    return mean_preds, lower_bound, upper_bound

n_bootstraps = 1000
ci = 95

# Split datasets
X_train_full = data.iloc[:22]
y_train = y.iloc[:22].values
X_test_full = data.iloc[22:28]
y_test = y.iloc[22:28].values
X_val_full = data.iloc[28:42]
y_val = y.iloc[28:42].values

scaler = StandardScaler()

for feature_set_name, features in feature_sets.items():
    X_train = X_train_full[features].values
    X_test = X_test_full[features].values
    X_val = X_val_full[features].values

    for model_name, hyperparams in tuned_hyperparameters[feature_set_name].items():
        if model_name == 'RF':
            model = RandomForestRegressor(**hyperparams)
            scale = False
        elif model_name == 'SVR':
            model = SVR(**hyperparams)
            scale = True
        elif model_name == 'XGB':
            model = XGBRegressor(**hyperparams)
            scale = False
        elif model_name == 'MLR':
            model = LinearRegression()
            scale = False

        if scale:
            X_train_scaled = scaler.fit_transform(X_train)
            X_test_scaled = scaler.transform(X_test)
            X_val_scaled = scaler.transform(X_val)
        else:
            X_train_scaled = X_train
            X_test_scaled = X_test
            X_val_scaled = X_val

        # Compute prediction intervals for train set
        train_preds, train_lower, train_upper = bootstrap_prediction_intervals(
            X_train_scaled, y_train, X_train_scaled, model, n_bootstraps=n_bootstraps, ci=ci
        )


        # Save test results
        train_df = pd.DataFrame({
            'y_true': y_train,
            'y_pred': train_preds,
            'lower_bound': train_lower,
            'upper_bound': train_upper
        })
        train_df['Feature Set'] = feature_set_name
        train_df['Model'] = model_name
        train_df.to_csv(f'./pred_interval_train_{feature_set_name}_{model_name}.csv', index=False)


        print(f"Saved prediction intervals for {model_name} with {feature_set_name}.")


Saved prediction intervals for RF with vi-qc.
Saved prediction intervals for SVR with vi-qc.
Saved prediction intervals for XGB with vi-qc.
Saved prediction intervals for RF with ga-qc.
Saved prediction intervals for SVR with ga-qc.
Saved prediction intervals for XGB with ga-qc.
Saved prediction intervals for MLR with ga-qc.
Saved prediction intervals for RF with vi-ms.
Saved prediction intervals for SVR with vi-ms.
Saved prediction intervals for XGB with vi-ms.
Saved prediction intervals for RF with ga-ms.
Saved prediction intervals for SVR with ga-ms.
Saved prediction intervals for XGB with ga-ms.
Saved prediction intervals for MLR with ga-ms.
